# Association Rule Mining

In [6]:
import pandas as pd
pd.set_option("max_colwidth", 150)

# Loading online retail dataset
f = "https://github.com/cs6220/cs6220.spring2019/raw/master/data/Online%20Retail.xlsx"
df = pd.read_excel(f)

# Transform transactions into baskets of items
basket = (df[df["Country"] == "United Kingdom"]
          .groupby(["InvoiceNo", "Description"])["Quantity"]
          .sum().unstack().reset_index().fillna(0)
          .set_index("InvoiceNo"))
# Convert counts to booleans
basket_sets = basket.applymap(lambda x: 1 if x >= 1 else 0)

## 1.1 Frequent Itemset Generation
### What are the top 5 1-itemsets with the highest support?

In [39]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(basket_sets, min_support=0.025, use_colnames=True)
frequent_itemsets
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets
frequent_one_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 1)]
frequent_one_itemsets
top_5_one_itemsets = frequent_one_itemsets.sort_values(by='support', ascending=False)
top_5_one_itemsets
top_5_one_itemsets = top_5_one_itemsets.head()
top_5_one_itemsets

,support,itemsets,length
123,0.098276,(WHITE HANGING HEART T-LIGHT HOLDER),1
54,0.087931,(JUMBO BAG RED RETROSPOT),1
99,0.076452,(REGENCY CAKESTAND 3 TIER),1
87,0.072323,(PARTY BUNTING),1
72,0.063158,(LUNCH BAG RED RETROSPOT),1


### What are the top 5 2-itemsets with the highest support?

In [40]:
frequent_two_itemsets = frequent_itemsets[(frequent_itemsets['length'] == 2)]
frequent_two_itemsets
top_5_two_itemsets = frequent_two_itemsets.sort_values(by='support', ascending=False).head()
top_5_two_itemsets

,support,itemsets,length
132,0.035617,"(JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKADOT)",2
130,0.031806,"(ROSES REGENCY TEACUP AND SAUCER , GREEN REGENCY TEACUP AND SAUCER)",2
134,0.031670,"(JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SUKI)",2
133,0.029809,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG RED RETROSPOT)",2
135,0.027541,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROSPOT)",2


### What is the highest support value for the 1-itemsets?

In [41]:
top_5_one_itemsets[:1]

,support,itemsets,length
123,0.098276,(WHITE HANGING HEART T-LIGHT HOLDER),1



### What is the highest support value for the 2-itemsets?


In [42]:
top_5_two_itemsets[:1]

,support,itemsets,length
132,0.035617,"(JUMBO BAG RED RETROSPOT, JUMBO BAG PINK POLKADOT)",2


## 1.2 Association Rule Generation
### What are the top 5 association rules?

In [50]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules = rules.sort_values(by='confidence', ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.031897,0.042377,0.026180,0.820768,19.368019,0.024828,5.342926
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.042377,0.043421,0.031806,0.750535,17.285056,0.029966,3.834527
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.043421,0.042377,0.031806,0.732497,17.285056,0.029966,3.579862
7,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.052586,0.087931,0.035617,0.677308,7.702719,0.030993,2.826438
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.039746,0.042196,0.025544,0.642694,15.231158,0.023867,2.680627


### What items make up one of the top association rules? Search online for the items (or at least items with the same name). Do you think they are likely to be bought together?

- Green regency teacup and saucer -> roses regency teacup and saucer are items that make up one of the top association rules
- I found the items online here: [green](https://www.rexlondon.com/green-regency-teacup-and-saucer) and [roses](https://www.rexlondon.com/roses-regency-teacup-and-saucer)
- I think these items are likely to be purchased together because those who enjoy tea enough to buy teacups and saucers are likely to purchase multiple sets for a collection. In addition, the earthy green set may be thought to compliment the roses set.
